Written BY Zetian


how we could deal with Job description and resume to give specific advise to a resume given by a user: Use BERT to encode the user’s resume alongside industry job descriptions and other resumes in the industry and measure their cosine similarity to identify alignment gaps.
Then extract the top TF–IDF keywords from the job descriptions that are missing in the user’s resume and suggest adding them as relevant skills or terms. 

In [2]:
import pandas as pd

In [ ]:
def get_user_category(system_jd_category_list, system_resume_category_list):


def get_user_resume():

    

In [ ]:
def analyze_keywords_similarity(jd_category, 
                                resume_category, 
                                user_resume, 
                                system_jd_keywords, 
                                system_resume_keywords):


In [ ]:
def analyze_embedding_similarity(jd_category, 
                                resume_category, 
                                user_resume, 
                                system_jd, 
                                system_resume):
    

In [ ]:
def main():

    system_jd = pd.read_csv('')
    system_resume = pd.read_csv('')
    system_jd_keywords = pd.read_csv('')
    system_resume_keywords = pd.read_csv('') 

    system_jd_category_list = system_jd['job'] #get a list of all existing jd_category
    system_resume_category_list = system_resume['Category'] #get a list of all existing resume_category


    jd_category, resume_category = get_user_category(system_jd_category_list, system_resume_category_list)
    # get the right category for user's resume to be analyze

    user_resume = get_user_resume()


    analyze_keywords_similarity(jd_category, resume_category, user_resume, system_jd_keywords, system_resume_keywords)
    #get the report of the similarity between user's resume and job desciprtion and other resumes in our database
    #in terms of keywords
    analyze_embedding_similarity(jd_category, resume_category, user_resume, system_jd, system_resume)
    #get the report of the similarity between user's resume and job desciprtion and other resumes in our database
    #in terms of embedding using BERT